In [10]:
import sys
import os
from pathlib import Path
from satpy import  Scene 
import matplotlib.pyplot as plt
from satpy.enhancements.enhancer import get_enhanced_image
# Retrocedemos un nivel desde 'notebooks' para llegar a la raíz del proyecto
root_path = Path(os.getcwd()).parent
sys.path.append(str(root_path / "src"))

# Ahora puedes importar tu config normalmente
from goes_processor import my_config_satpy

# Verificación de carga de archivos (basado en tu archivo config_satpy.py)
print(f"Ruta de búsqueda Satpy: {my_config_satpy.satpy.config.get('config_path')}")


# Celda 2: Ruta al archivo (cambia si es necesario)
file_path = "OR_ABI-L2-FDCF-M6_G19_s20260031200230_e20260031209539_c20260031210078.nc"
base_name = Path(file_path).stem
# Verificar que existe
if not Path(file_path).exists():
    print("Archivo no encontrado. Verifica la ruta.")
else:
    print(f"Archivo encontrado: {file_path}")

scn = Scene(filenames=[file_path ], reader='abi_l2_nc')
scn.load(['my_fdc_fn03'])


# Genera la imagen realzada
img = get_enhanced_image(scn['my_fdc_fn03'])

# Muestra la imagen directamente (esto suele abrir el visor de imágenes del sistema 
# o mostrarlo en el notebook dependiendo de la configuración)
img.show()

Ruta de búsqueda Satpy: ['/home/legion/bulk/MAIE_tesis2026/f01_code/MAIE_tesis_github/src/goes_processor/satpy_configs']
Archivo encontrado: OR_ABI-L2-FDCF-M6_G19_s20260031200230_e20260031209539_c20260031210078.nc


In [11]:
# Extraer la imagen ya con la rampa de colores aplicada
img = get_enhanced_image(scn['my_fdc_fn02']).convert("RGB")

# Mostrar en el notebook
plt.figure(figsize=(12, 12))
plt.imshow(img.data.transpose('y', 'x', 'bands') / 255.0) # Normalización para plt
plt.title(f"Experimento FN02 - Escala Magma\n{base_name}", color='white')
plt.axis('off')
plt.show()

KeyError: "No dataset matching 'DataQuery(name='my_fdc_fn02')' found"

In [ ]:
# Celda 1: Importaciones necesarias
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pathlib import Path

In [ ]:
# Celda 2: Ruta al archivo (cambia si es necesario)
file_path = "OR_ABI-L2-FDCF-M6_G19_s20260031200230_e20260031209539_c20260031210078.nc"

# Verificar que existe
if not Path(file_path).exists():
    print("Archivo no encontrado. Verifica la ruta.")
else:
    print(f"Archivo encontrado: {file_path}")

In [ ]:
# Celda 3: Abrir el archivo con xarray (formato NetCDF)
ds = xr.open_dataset(file_path)
print("Dataset cargado correctamente.")
print("\nInformación general del archivo:")
print(ds)

In [ ]:
# Celda 4: Ver todas las variables disponibles (las más importantes de FDCF)
print("Variables disponibles en el archivo:")
for var in ds.variables:
    print(f" - {var}: {ds[var].attrs.get('long_name', 'sin descripción')}")

In [ ]:
# Celda 5: Inspeccionar algunas variables clave de detección de fuego
print("\nResumen de variables importantes:")
print(ds['Power'].attrs)          # Potencia radiativa del fuego (MW)
print(ds['Temp'])                 # Temperatura de brillo (K)
print(ds['Area'])                 # Área del fuego (m²)
print(ds['Mask'])                 # Máscara de detección (0=no fuego, 10-15=fuego confirmado)

In [ ]:
# Celda 6: Plot simple de la máscara de fuego (donde hay detección)
plt.figure(figsize=(10, 8))
ds['Mask'].plot(cmap='Reds', add_colorbar=True)
plt.title("Máscara de detección de fuego (Mask)")
plt.show()

In [ ]:
# Celda 7: Plot de potencia radiativa (solo donde hay fuego)
power = ds['Power'].where(ds['Mask'] >= 10)  # Solo valores con fuego confirmado
plt.figure(figsize=(10, 8))
power.plot(cmap='hot', add_colorbar=True, robust=True)
plt.title("Potencia Radiativa de incendios (MW)")
plt.show()

In [ ]:
# Celda 8: Plot con mapa (proyección geográfica)
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='50m')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')

# Plot de potencia (solo fuego)
power.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='hot',
    cbar_kwargs={'label': 'Potencia Radiativa (MW)'},
    robust=True
)

plt.title("Detección de incendios GOES-19 - Potencia Radiativa")
plt.show()

In [ ]:
# Celda 9: Estadísticas básicas de fuego
fire_mask = ds['Mask'] >= 10
num_fires = fire_mask.sum().values
total_power = ds['Power'].where(fire_mask).sum().values
avg_temp = ds['Temp'].where(fire_mask).mean().values

print(f"Número de píxeles con fuego detectado: {num_fires}")
print(f"Potencia radiativa total: {total_power:.2f} MW")
print(f"Temperatura media de fuego: {avg_temp:.2f} K ({avg_temp - 273.15:.2f} °C)")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr

# 1. Tu Diccionario Maestro WFABBA (61 Categorías)
# He mantenido tus etiquetas y colores hexadecimales
ABI_WFABBA_FIRE_MASK = {
    -99: {"label": "Initialization value", "color": "#9e9e9e"},
    0: {"label": "Non-processed region", "color": "#cfd8dc"},
    10: {"label": "Processed fire pixel", "color": "#ff0000"},
    11: {"label": "Saturated fire pixel", "color": "#ff0000"},
    12: {"label": "Cloud contaminated fire pixel", "color": "#fb8c00"},
    13: {"label": "High probability fire pixel", "color": "#ffd600"},
    14: {"label": "Medium probability fire pixel", "color": "#ffea00"},
    15: {"label": "Low probability fire pixel", "color": "#fff59d"},
    20: {"label": "Reserved", "color": "#eeeeee"},
    21: {"label": "Reserved", "color": "#eeeeee"},
    22: {"label": "Reserved", "color": "#eeeeee"},
    23: {"label": "Reserved", "color": "#eeeeee"},
    24: {"label": "Reserved", "color": "#eeeeee"},
    25: {"label": "Reserved", "color": "#eeeeee"},
    30: {"label": "TF Processed fire pixel", "color": "#d32f2f"},
    31: {"label": "TF Saturated fire pixel", "color": "#880e4f"},
    32: {"label": "TF Cloud contaminated fire pixel", "color": "#ef6c00"},
    33: {"label": "TF High probability fire pixel", "color": "#f9a825"},
    34: {"label": "TF Medium probability fire pixel", "color": "#fdd835"},
    35: {"label": "TF Low probability fire pixel", "color": "#fff176"},
    40: {"label": "Space pixel", "color": "#000000"},
    50: {"label": "Zenith angle block-out (>80°)", "color": "#1976d2"},
    60: {"label": "Glint/Solar zenith block-out", "color": "#64b5f6"},
    100: {"label": "Processed region of image", "color": "#4caf50"},
    120: {"label": "Missing 3.9 µm (Ch 7)", "color": "#9e9e9e"},
    121: {"label": "Missing 11.2 µm (Ch 14)", "color": "#9e9e9e"},
    123: {"label": "Saturation 3.9 µm (Ch 7)", "color": "#9e9e9e"},
    124: {"label": "Saturation 11.2 µm (Ch 14)", "color": "#9e9e9e"},
    125: {"label": "Invalid reflectivity input", "color": "#9e9e9e"},
    126: {"label": "Unusable Ch 7 < 200 K", "color": "#9e9e9e"},
    127: {"label": "Unusable Ch 14 < 200 K", "color": "#9e9e9e"},
    130: {"label": "Reserved", "color": "#eeeeee"},
    150: {"label": "Invalid ecosystem type", "color": "#6d4c41"},
    151: {"label": "Sea water", "color": "#0277bd"},
    152: {"label": "Coastline fringe", "color": "#4db6ac"},
    153: {"label": "Inland water / mix", "color": "#26a69a"},
    155: {"label": "Reserved", "color": "#eeeeee"},
    160: {"label": "Invalid emissivity value", "color": "#8d6e63"},
    170: {"label": "No background computed", "color": "#795548"},
    180: {"label": "Error T to Radiance", "color": "#5d4037"},
    182: {"label": "Error adjusted T to Radiance", "color": "#5d4037"},
    185: {"label": "Dozier bisection invalid", "color": "#4e342e"},
    186: {"label": "Invalid radiances (Newton)", "color": "#4e342e"},
    187: {"label": "Newton method errors", "color": "#4e342e"},
    188: {"label": "Error computing Dozier area", "color": "#4e342e"},
    200: {"label": "11.2 µm cloud test", "color": "#90a4ae"},
    201: {"label": "Neg diff & ≤273 K test", "color": "#90a4ae"},
    205: {"label": "Neg diff cloud test", "color": "#90a4ae"},
    210: {"label": "Pos diff cloud test", "color": "#90a4ae"},
    215: {"label": "Albedo cloud test", "color": "#90a4ae"},
    220: {"label": "12.3 µm cloud test", "color": "#90a4ae"},
    225: {"label": "11.2–12.3 µm neg diff test", "color": "#90a4ae"},
    230: {"label": "11.2–12.3 µm pos diff test", "color": "#90a4ae"},
    240: {"label": "Along-scan refl (3.9 µm)", "color": "#78909c"},
    245: {"label": "Along-scan refl (albedo)", "color": "#78909c"},
}

# --- PROCESO: CONVERSIÓN HEX A RGB (0-255) ---
print(f"{'ID':<5} | {'RGB (0-255)':<15} | {'Label'}")
print("-" * 80)

for code in sorted(ABI_WFABBA_FIRE_MASK.keys()):
    info = ABI_WFABBA_FIRE_MASK[code]
    # Conversión
    rgb_dec = mcolors.to_rgb(info['color'])
    rgb_255 = tuple(int(c * 255) for c in rgb_dec)
    
    # Guardar en el diccionario para uso posterior
    info['rgb_255'] = rgb_255
    
    print(f"{code:<5} | {str(rgb_255):<15} | {info['label']}")

# 2. Preparar imagen RGBA
mask_data = ds.Mask.values
h, w = mask_data.shape
rgba_img = np.zeros((h, w, 4))

# Color de fondo para IDs no definidos (Gris oscuro)
rgba_img[:] = mcolors.to_rgba('#121212')

# 3. Aplicar colores iterando sobre tu diccionario
for code, info in ABI_WFABBA_FIRE_MASK.items():
    mask_indices = (mask_data == code)
    rgba_img[mask_indices] = mcolors.to_rgba(info['color'])

# 4. Plotear Disco Completo
plt.figure(figsize=(16, 16), facecolor='black')

plt.imshow(rgba_img, interpolation='nearest')

plt.title(f"WFABBA Fire Mask Classification (v.0.0.1)\n{ds.attrs.get('dataset_name', 'GOES-19 ABI')}", 
          fontsize=18, color='white', pad=25)

plt.axis('off')
plt.show()

# 5. Resumen de Fuego
fuegos = np.isin(mask_data, [10, 11, 12, 13, 14, 15, 30, 31, 32, 33, 34, 35]).sum()
print(f"🔥 Total de píxeles detectados como fuego (Normal + Filtrado Temporal): {fuegos:,}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr

# 1. Diccionario Maestro WFABBA - Versión "Atmospheric Glow" v.0.0.4
ABI_WFABBA_FIRE_MASK = {
    # Fondo y No Procesado
    -99: {"label": "Initialization value", "color": "#121212"},
    0: {"label": "Non-processed region", "color": "#0A121E"}, # Azul muy oscuro

    # Fuegos (Escala de calor)
    10: {"label": "Processed fire pixel", "color": "#FF0000"},
    11: {"label": "Saturated fire pixel", "color": "#FFFF00"}, 
    12: {"label": "Cloud contaminated fire pixel", "color": "#FF8C00"},
    13: {"label": "High probability fire pixel", "color": "#FF4500"},
    14: {"label": "Medium probability fire pixel", "color": "#FFA500"},
    15: {"label": "Low probability fire pixel", "color": "#FFD700"},

    # --- EFECTOS ESPECIALES DE BORDE Y BLOQUEO ---
    40: {"label": "Space pixel", "color": "#000000"},           # Negro puro (Espacio)
    50: {"label": "Atmospheric Limb (>80°)", "color": "#FF007F"}, # ROSA FUCSIA (Borde)
    60: {"label": "Solar Glint/Block-out", "color": "#00FFC8"},   # CIAN ELÉCTRICO (Bloqueo)

    # Fuegos Filtrados Temporales (Púrpuras)
    30: {"label": "TF Processed fire pixel", "color": "#D500F9"},
    31: {"label": "TF Saturated fire pixel", "color": "#651FFF"},
    32: {"label": "TF Cloud contaminated fire pixel", "color": "#3D5AFE"},
    33: {"label": "TF High probability fire pixel", "color": "#00B0FF"},
    34: {"label": "TF Medium probability fire pixel", "color": "#00E5FF"},
    35: {"label": "TF Low probability fire pixel", "color": "#1DE9B6"},

    # Superficies
    100: {"label": "Processed region (Land)", "color": "#143311"}, # Verde oscuro
    151: {"label": "Sea water", "color": "#003366"},
    152: {"label": "Coastline fringe", "color": "#005B96"},
    153: {"label": "Inland water", "color": "#0077BE"},

    # Nubes y Otros (Grises suaves)
    200: {"label": "Cloud test", "color": "#CFD8DC"},
    215: {"label": "Cloud test day", "color": "#ECEFF1"},
}

# Rellenar códigos faltantes de nubes y errores para que el script no falle
for i in range(16, 26): ABI_WFABBA_FIRE_MASK[i] = {"label": "Reserved", "color": "#1B263B"}
for i in range(120, 131): ABI_WFABBA_FIRE_MASK[i] = {"label": "Bad input", "color": "#2F2F2F"}
for i in [201, 205, 210, 220, 225, 230, 240, 245]:
    if i not in ABI_WFABBA_FIRE_MASK:
        ABI_WFABBA_FIRE_MASK[i] = {"label": "Cloud test", "color": "#7F8C8D"}

# --- CONVERSIÓN RGB 0-255 ---
print(f"{'ID':<5} | {'RGB (0-255)':<15} | {'Label'}")
print("-" * 80)
for code in sorted(ABI_WFABBA_FIRE_MASK.keys()):
    info = ABI_WFABBA_FIRE_MASK[code]
    rgb_dec = mcolors.to_rgb(info['color'])
    info['rgb_255'] = tuple(int(c * 255) for c in rgb_dec)
    print(f"{code:<5} | {str(info['rgb_255']):<15} | {info['label']}")

# 2. Generar Imagen
mask_data = ds.Mask.values
rgba_img = np.zeros((mask_data.shape[0], mask_data.shape[1], 4))
rgba_img[:] = mcolors.to_rgba("#3450EC")

for code, info in ABI_WFABBA_FIRE_MASK.items():
    rgba_img[mask_data == code] = mcolors.to_rgba(info['color'])

# 3. Plot
plt.figure(figsize=(16, 16), facecolor='black')
plt.imshow(rgba_img, interpolation='nearest')

plt.title(f"GOES-19 FDCF - WFABBA Atmospheric Edge & Block-out\nv.0.0.4", 
          fontsize=22, color='#FF007F', pad=30, fontweight='bold')

plt.axis('off')
plt.show()

# 5. Resumen de Fuego
fuegos = np.isin(mask_data, [10, 11, 12, 13, 14, 15, 30, 31, 32, 33, 34, 35]).sum()
print(f"🔥 Total de píxeles detectados como fuego (Normal + Filtrado Temporal): {fuegos:,}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr

# 1. Diccionario Maestro WFABBA v.0.0.6 (Estadística de Persistencia)
ABI_WFABBA_FIRE_MASK = {
    # Fondo y Océano
    -99: {"label": "Initialization", "color": "#3450EC"},
    0:   {"label": "Ocean / Deep Water", "color": "#001B3D"},

    # --- FUEGOS NUEVOS (PRIMERA DETECCIÓN) ---
    10:  {"label": "New Fire: Processed", "color": "#FF0000"},
    11:  {"label": "New Fire: Saturated", "color": "#FFFF00"}, 
    12:  {"label": "New Fire: Cloud Contam.", "color": "#FF8C00"},
    13:  {"label": "New Fire: High Prob.", "color": "#FF4500"},
    14:  {"label": "New Fire: Med Prob.", "color": "#FFA500"},
    15:  {"label": "New Fire: Low Prob.", "color": "#FFD700"},

    # --- FUEGOS CONFIRMADOS (PERSISTENTES / TF) ---
    30:  {"label": "Persistent Fire: Processed", "color": "#D500F9"},
    31:  {"label": "Persistent Fire: Saturated", "color": "#D500F9"},
    32:  {"label": "Persistent Fire: Cloud Contam.", "color": "#D500F9"},
    33:  {"label": "Persistent Fire: High Prob.", "color": "#D500F9"},
    34:  {"label": "Persistent Fire: Med Prob.", "color": "#D500F9"},
    35:  {"label": "Persistent Fire: Low Prob.", "color": "#D500F9"},

    # Efectos Especiales
    40:  {"label": "Space", "color": "#000000"},
    50:  {"label": "Atmospheric Limb (Rosa)", "color": "#FF007F"},
    60:  {"label": "Solar Block-out (Cian)", "color": "#00FFC8"},

    # Superficies y Nubes
    100: {"label": "Land / Forest", "color": "#1A3314"},
    151: {"label": "Sea Water", "color": "#003399"},
    153: {"label": "Inland Water", "color": "#0077BE"},
    200: {"label": "Cloud Test", "color": "#90A4AE"},
}

# (Omitimos el relleno de nubes por brevedad, pero mantenlo en tu script)

# 2. PROCESAMIENTO
mask_data = ds.Mask.values
rgba_img = np.zeros((mask_data.shape[0], mask_data.shape[1], 4))
rgba_img[:] = mcolors.to_rgba('#3450EC')

for code, info in ABI_WFABBA_FIRE_MASK.items():
    rgba_img[mask_data == code] = mcolors.to_rgba(info['color'])

# 3. PLOT
plt.figure(figsize=(16, 16), facecolor='black')
plt.imshow(rgba_img, interpolation='nearest')
plt.title(f"GOES-19 FDCF - Fire Persistence Analysis v.0.0.6", 
          fontsize=22, color='#FF007F', pad=30, fontweight='bold')
plt.axis('off')
plt.show()

# 4. --- AUDITORÍA ESTADÍSTICA DE PERSISTENCIA ---
ids_fuego_nuevo = [10, 11, 12, 13, 14, 15]
ids_fuego_persistente = [30, 31, 32, 33, 34, 35]

count_nuevo = np.isin(mask_data, ids_fuego_nuevo).sum()
count_persistente = np.isin(mask_data, ids_fuego_persistente).sum()
total_fuegos = count_nuevo + count_persistente

print(f"\n" + "="*40)
print(f"🔥 REPORTE DE INCENDIOS (WFABBA)")
print(f"="*40)
print(f"Fuegos Nuevos (Rojos/Am):       {count_nuevo:>8,}")
print(f"Fuegos Persistentes (Púr/Az):  {count_persistente:>8,}")
print(f"-"*40)
print(f"TOTAL DE PÍXELES ACTIVOS:      {total_fuegos:>8,}")

if total_fuegos > 0:
    ratio_p = (count_persistente / total_fuegos) * 100
    print(f"Índice de Persistencia:        {ratio_p:.1f}%")
print(f"="*40)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr

# 1. CARGAR DATOS (Asumiendo que tienes ds_fire para la máscara y ds_abi para las bandas)
# Si estás usando el mismo archivo para todo, usa ds.
mask_data = ds.Mask.values

# 2. CREAR LA CAPA DE FUEGO (RGBA)
h, w = mask_data.shape
fire_rgba = np.zeros((h, w, 4)) # 4 canales: R, G, B, Alpha

# Definimos solo los IDs que queremos que "brillen" sobre la imagen
# Usamos los colores de tu v.0.0.6
for code, info in ABI_WFABBA_FIRE_MASK.items():
    # Solo damos opacidad a Fuegos, Bordes y Bloqueos
    # El océano y la tierra del FDCF los dejamos transparentes para ver el RGB de fondo
    if (10 <= code <= 35) or code in [50, 60]:
        mask_indices = (mask_data == code)
        fire_rgba[mask_indices] = mcolors.to_rgba(info['color'], alpha=1.0)
    else:
        # Todo lo demás (agua, tierra, nubes del FDCF) es invisible
        mask_indices = (mask_data == code)
        fire_rgba[mask_indices] = [0, 0, 0, 0] 

# 3. CARGAR TU COMPOSITE (Ejemplo: True Color)
# Aquí pondrías el array de tu imagen RGB ya procesada (0 a 1)
# Si no lo tienes a mano, usaremos un fondo negro para el ejemplo
rgb_background = np.zeros((h, w, 3)) 

# 4. PLOTEAR SUPERPUESTO
plt.figure(figsize=(16, 16), facecolor='black')

# Primero la imagen satelital real
plt.imshow(rgb_background, interpolation='nearest')

# Encima la máscara de fuego (el canal alpha hace la magia)
plt.imshow(fire_rgba, interpolation='nearest')

plt.title("GOES-19 ABI Enhanced Fire Composite\nFDCF Mask Overlay v.0.0.7", 
          color='white', fontsize=20, pad=25)
plt.axis('off')
plt.show()

In [ ]:
import sys
import os
from pathlib import Path
from satpy import Scene
from pyresample import AreaDefinition

# --- 1. CONFIGURACIÓN DE RUTAS ---
current_dir = Path(os.getcwd())
root_path = current_dir if "src" not in current_dir.parts else current_dir.parent
sys.path.append(str(root_path / "src"))

# Carga de tu config (Caché y YAMLs)
try:
    from goes_processor import my_config_satpy
    print("✅ Sistema de configuración Satpy v.0.0.1 activado.")
except ImportError:
    os.environ['SATPY_CONFIG_PATH'] = str(root_path / "src" / "goes_processor" / "satpy_configs")
    print("⚠️ Usando ruta de configuración manual.")

# --- 2. DEFINICIÓN DE ÁREA GLOBAL WGS84 (3600 x 1800) ---
# Esta definición crea un mapa equirrectangular del mundo entero
area_id = 'global_wgs84_3600'
description = 'Mapa Global WGS84 0.1 deg'
proj_id = 'epsg4326'
projection = {'proj': 'longlat', 'datum': 'WGS84', 'no_defs': True}
width = 3600 
height = 1800
# Extensión global: [oeste, sur, este, norte]
area_extent = [-180.0, -90.0, 180.0, 90.0] 

global_area = AreaDefinition(area_id, description, proj_id, projection, width, height, area_extent)

# --- 3. PROCESAMIENTO ---
file_path = "OR_ABI-L2-FDCF-M6_G19_s20260031200230_e20260031209539_c20260031210078.nc"

if not Path(file_path).exists():
    print(f"❌ Archivo no encontrado: {file_path}")
    sys.exit()

print(f"🚀 Procesando FDCF Global (3600x1800)...")

scn = Scene(filenames=[file_path], reader='abi_l2_nc')
scn.load(['fdcf_mask'])

# A. Guardar Original (Fixed Grid - Geostacionario)
print("📸 Guardando vista original (Sistema Geostacionario)...")
scn.save_dataset('fdcf_mask', filename='fuego_original_v001.png')

# B. Reproyección Global (3600 x 1800)
print(f"🗺️ Reproyectando a malla global WGS84...")
resampled_scn = scn.resample(global_area)

# C. Exportar Resultados Finales
print("💾 Exportando archivos finales...")

# PNG Global
resampled_scn.save_dataset('fdcf_mask', filename='fuego_global_3600.png')

# GeoTIFF Global para GIS
resampled_scn.save_dataset(
    'fdcf_mask', 
    filename='fuego_global_3600.tif', 
    writer='geotiff'
)

print("-" * 50)
print("✅ PROCESO COMPLETADO")
print(f"Dimensiones: {width}x{height}")
print(f"Archivos: fuego_global_3600.png/tif")
print("-" * 50)

In [ ]:
scn.show('fdcf_mask')

In [ ]:
import sys
import os
from pathlib import Path
from satpy import Scene
from pyresample import AreaDefinition
# Corrección de la importación
from satpy.enhancements.enhancer import get_enhanced_image

# --- Configuración de Rutas ---
current_dir = Path(os.getcwd())
root_path = current_dir if "src" not in current_dir.parts else current_dir.parent
sys.path.append(str(root_path / "src"))

# Activar tu configuración v.0.0.1
from goes_processor import my_config_satpy

# --- Definición de Área Global (3600x1800) ---
global_area = AreaDefinition(
    'global_wgs84', 'Global 0.1deg', 'epsg4326',
    {'proj': 'longlat', 'datum': 'WGS84', 'no_defs': True},
    3600, 1800, [-180.0, -90.0, 180.0, 90.0]
)

# --- Carga y Procesamiento ---
file_path = "OR_ABI-L2-FDCF-M6_G19_s20260031200230_e20260031209539_c20260031210078.nc"
scn = Scene(filenames=[file_path], reader='abi_l2_nc')

# CARGAR PRODUCTO (Asegúrate de haber actualizado el composite a fdcf_mask)
scn.load(['fdcf_mask'])

# 1. Mostrar en pantalla con COLORES
print("📺 Abriendo visor de colores...")
# Forzamos el realce usando tu configuración YAML
img = get_enhanced_image(scn['fdcf_mask'])
img.show()

# 2. Guardar Disco Original con COLORES
print("📸 Guardando disco original...")
scn.save_dataset('fdcf_mask', filename='fuego_original_color.png')

# 3. Reproyectar y Guardar Global
print("🗺️ Reproyectando a Global 3600x1800...")
resampled_scn = scn.resample(global_area)

print("💾 Guardando GeoTIFF y PNG Global...")
resampled_scn.save_dataset('fdcf_mask', filename='fuego_global_3600.png')
resampled_scn.save_dataset('fdcf_mask', filename='fuego_global_3600.tif', writer='geotiff')

print("✅ Proceso v.0.0.1 finalizado con éxito.")

In [ ]:
# Carga normal
scn.load(['fdcf_mask'])

# Generar imagen realzada
img = get_enhanced_image(scn['fdcf_mask'])

# Si aún ves transparencia en el visor, fuerza a que la imagen sea RGB pura (sin alfa)
img = img.convert("RGB") 

img.show()

In [ ]:
# ... después de scn.load(['fdcf_mask']) ...

# 1. Extraemos el dataset
data = scn['fdcf_mask']

# 2. Rellenamos los NANs (transparentes) con el valor 40 (Negro/Espacio)
# Esto "borra" la transparencia antes de aplicar el color
data = data.fillna(40)

# 3. Lo devolvemos al scene para que Satpy use el dato "sólido"
scn['fdcf_mask'] = data

# 4. Ahora generamos la imagen
from satpy.enhancements.enhancer import get_enhanced_image
img = get_enhanced_image(scn['fdcf_mask'])

# 5. Forzamos la eliminación definitiva del canal Alfa
img = img.convert("RGB")

img.show()

In [ ]:
import sys
import os
import json
import yaml
import numpy as np
from pathlib import Path
from satpy import Scene
from pyresample import AreaDefinition
from satpy.enhancements.enhancer import get_enhanced_image

# --- 1. CONFIGURACIÓN DE RUTAS (Basado en tu estructura en legion) ---
# root_path será: /home/legion/bulk/MAIE_tesis2026/f01_code/MAIE_tesis_github
root_path = Path("/home/legion/bulk/MAIE_tesis2026/f01_code/MAIE_tesis_github")
sys.path.append(str(root_path / "src"))

# Ruta absoluta al archivo de configuración de colores
config_path = root_path / "src" / "goes_processor" / "satpy_configs" / "enhancements" / "abi.yaml"

# --- 2. FUNCIÓN PARA LEER COLORES DEL YAML ---
def extraer_escala_del_yml(path):
    if not path.exists():
        return {"error": f"Archivo no encontrado en {path}"}
    
    with open(path, 'r') as f:
        # Usamos unsafe_load porque el YAML de Satpy tiene tags especiales de Python
        conf = yaml.unsafe_load(f)
        
    try:
        # Buscamos el bloque de fdcf_mask en el YAML
        enhance_node = conf['enhancements']['fdcf_mask']['operations']
        # Buscamos la operación 'colorize' o la primera que tenga palettes
        for op in enhance_node:
            if 'kwargs' in op and 'palettes' in op['kwargs']:
                data = op['kwargs']['palettes'][0]
                colores = data['colors']
                valores = data['values']
                return {str(val): col for val, col in zip(valores, colores)}
    except Exception as e:
        return {"error": f"Error parseando YAML: {str(e)}"}
    return {"error": "Estructura de colores no encontrada"}

# --- 3. CARGA Y PROCESAMIENTO ---
from goes_processor import my_config_satpy

# Asegúrate de que este archivo esté en la raíz o pon la ruta completa
file_path = "OR_ABI-L2-FDCF-M6_G19_s20260031200230_e20260031209539_c20260031210078.nc"
scn = Scene(filenames=[file_path], reader='abi_l2_nc')
scn.load(['fdcf_mask'])

# ELIMINACIÓN DE TRANSPARENCIA (Lógica de Matplotlib - Fuerza Bruta)
data = scn['fdcf_mask']
# Reemplazamos NaNs y el valor 127 por 40 (que es tu fondo sólido en el YAML)
data_filled = data.where(~np.isnan(data), 40).where(data != 127, 40)

# Limpiar metadatos que obligan a Satpy a ocultar píxeles
for attr in ['_FillValue', 'valid_range']:
    if attr in data_filled.attrs:
        del data_filled.attrs[attr]

scn['fdcf_mask'] = data_filled

# --- 4. GENERAR PRODUCTOS ---
print("📸 Generando PNG Original...")
img_orig = get_enhanced_image(scn['fdcf_mask']).convert("RGB")
img_orig.save('fuego_original_v001.png')

# Reproyección Global WGS84 (3600x1800)
print("🗺️ Reproyectando a WGS84 Global...")
global_area = AreaDefinition(
    'global_wgs84', 'Global 0.1deg', 'epsg4326',
    {'proj': 'longlat', 'datum': 'WGS84', 'no_defs': True},
    3600, 1800, [-180.0, -90.0, 180.0, 90.0]
)
resampled_scn = scn.resample(global_area)

print("💾 Guardando archivos WGS84...")
img_wgs84 = get_enhanced_image(resampled_scn['fdcf_mask']).convert("RGB")
img_wgs84.save('fuego_global_3600_v001.png')
resampled_scn.save_dataset('fdcf_mask', filename='fuego_global_3600_v001.tif', writer='geotiff')

# --- 5. METADATOS Y ESCALA ---
escala_detectada = extraer_escala_del_yml(config_path)

detalles = {
    "version": "v.0.0.1",
    "archivo_fuente": file_path,
    "escala_detectada_desde_yml": escala_detectada,
    "config_path_usado": str(config_path),
    "geometria": {
        "ancho": 3600,
        "alto": 1800,
        "proyeccion": "EPSG:4326 (WGS84)"
    },
    "procesamiento": "Relleno de NaNs/127 con ID 40 y conversion RGB para eliminar transparencia"
}

with open('detalles_proceso_v001.json', 'w') as jfile:
    json.dump(detalles, jfile, indent=4)

print("-" * 30)
print("✅ v.0.0.1 Finalizada con éxito.")
print(f"📁 Escala leída de: {config_path}")
print("-" * 30)

In [ ]:
import sys
import os
from pathlib import Path
from satpy import Scene
import matplotlib.pyplot as plt
from satpy.enhancements.enhancer import get_enhanced_image

# Asegúrate de que Python encuentre tu carpeta src
sys.path.append(str(Path(os.getcwd()).parent / "src"))

# Importamos las funciones de tu lógica modular
from goes_processor.processing.logic_how.fdc import fn02_process_analysis, fn_get_color_scale_config

In [ ]:
# Configuración de rutas
file_nc = "OR_ABI-L2-FDCF-M6_G19_s20260031200230_e20260031209539_c20260031210078.nc"
output_test = Path("./test_outputs")
output_test.mkdir(exist_ok=True)

# 1. Cargar Escena
scn = Scene(filenames=[file_nc], reader='abi_l2_nc')
scn.load(['my_fdcf_fn01'])

# 2. Resampling rápido para el notebook (opcional, puedes usar el nativo)
# scn_res = scn.resample(resampler='native') 

# 3. Ejecutar FN02 (Simulamos los argumentos que pide tu función)
base_name = Path(file_nc).stem
# Nota: Pasamos scn tanto para la data nativa como para la resampleada si no necesitas WGS84 ahora
fn02_process_analysis(scn, scn, output_test, base_name, start_ts=0)

print(f"✅ Procesamiento fn02 completado en {output_test}")